In [1]:
from keras.models import load_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout
import numpy as np
import cv2
import os
from numpy import asarray

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [13]:
encoder = load_model('/content/drive/MyDrive/Models/enc6.h5', custom_objects={'Sampling': Sampling})
generator = load_model('/content/drive/MyDrive/Models/gen6.h5')

In [5]:
def read_images(path, isNormalized, width, height):
  i = 0
  instances = []
  for image in os.listdir(path):
    i += 1
    img = cv2.imread(path+image)
    output = cv2.resize(img, (width,height))
    if isNormalized == True :
      normalizedImg = np.zeros((width, height))
      normalizedImg = cv2.normalize(output,  normalizedImg, 0, 1, cv2.NORM_MINMAX)
      numpydata = asarray(normalizedImg)
    else:
      numpydata = asarray(output)
    instances.append(numpydata)  
    if i % 500 == 0 : print(i)
  return np.array(instances)

In [14]:
real_data = read_images('/content/drive/MyDrive/AGB_224/', False, 224, 224)
real_data.shape

500
1000
1500
2000


(2413, 224, 224, 3)

In [15]:
batch_size = 64
encoder.trainable = False 
generator.trainable = False 
counter = 0
for i in range(0,len(real_data) // batch_size):
  tmp_z = encoder(real_data[i*batch_size:(i+1)*batch_size])
  z = np.array(tmp_z)[2]
  generated = generator.predict_on_batch(z)
  for j in range(0,batch_size):
    counter += 1
    if counter % 500 == 0:
      print(counter)
    cv2.imwrite('/content/drive/MyDrive/generated_6/image_'+str(counter)+'.jpg', ((generated[j]+ 1)* 127).astype(np.uint8))

500
1000
1500
2000


In [11]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data


# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

model = InceptionV3(include_top=False, pooling='avg', input_shape=(75,75,3))

In [12]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/generated_2/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:2300], images2[:2300])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
1000
1500
2000
Loaded:  (2413, 75, 75, 3) (2368, 75, 75, 3)
72/72 [==============================] - 1s 14ms/step
FID: 417.510


In [16]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/generated_3/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:2300], images2[:2300])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
1000
1500
2000
Loaded:  (2413, 75, 75, 3) (2368, 75, 75, 3)
72/72 [==============================] - 1s 15ms/step
FID: 761.287


In [18]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/generated_4/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:2300], images2[:2300])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
1000
1500
2000
Loaded:  (2413, 75, 75, 3) (2368, 75, 75, 3)
72/72 [==============================] - 1s 14ms/step
FID: 448.770


In [19]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/generated_5/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:2300], images2[:2300])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
1000
1500
2000
Loaded:  (2413, 75, 75, 3) (2368, 75, 75, 3)
72/72 [==============================] - 1s 20ms/step
FID: 407.834


In [20]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/generated_6/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:2300], images2[:2300])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
1000
1500
2000
Loaded:  (2413, 75, 75, 3) (2368, 75, 75, 3)
72/72 [==============================] - 1s 14ms/step
FID: 632.158


In [21]:
real_images = read_images('/content/drive/MyDrive/AGB_224/', False, 75, 75)
generated_images = read_images('/content/drive/MyDrive/GAN/', False, 75, 75)
print('Loaded: ', real_images.shape, generated_images.shape)
images1 = real_images.astype('float32')
images2 = generated_images.astype('float32')
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1[:700], images2[:700])
print('FID: %.3f' % fid)

500
1000
1500
2000
500
Loaded:  (2413, 75, 75, 3) (707, 75, 75, 3)
22/22 [==============================] - 1s 22ms/step
FID: 99.963


In [22]:
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from numpy import asarray


# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# pre-process images, scale to [-1,1]
		subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std


generated_images = read_images('/content/drive/MyDrive/GAN/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
3/3 [==============================] - 0s 114ms/step
score 2.2334046 0.1424676


In [12]:
generated_images = read_images('/content/drive/MyDrive/generated_2/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
1000
1500
2000
8/8 [==============================] - 1s 106ms/step
score 1.4731474 0.23376875


In [13]:
generated_images = read_images('/content/drive/MyDrive/generated_3/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
1000
1500
2000
8/8 [==============================] - 1s 101ms/step
score 1.5378238 0.24459374


In [14]:
generated_images = read_images('/content/drive/MyDrive/generated_4/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
1000
1500
2000
8/8 [==============================] - 1s 102ms/step
score 1.500274 0.120681375


In [23]:
generated_images = read_images('/content/drive/MyDrive/generated_5/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
1000
1500
2000
8/8 [==============================] - 1s 114ms/step
score 1.514185 0.17816958


In [24]:
generated_images = read_images('/content/drive/MyDrive/generated_6/', False, 299, 299)
is_avg, is_std = calculate_inception_score(generated_images)
print('score', is_avg, is_std)

500
1000
1500
2000
8/8 [==============================] - 1s 113ms/step
score 1.4316797 0.17574733
